In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate

import imageio

import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis

In [ ]:
# image_name = '00000B94.png'
# image_path = pathlib.Path('.').parent.joinpath('data', 'images', 'lossless', image_name)

# if not image_path.exists():
#     url = f'https://zenodo.org/record/3520266/files/{image_name}?download=1'
#     urllib.request.urlretrieve(url, image_path)

In [ ]:
degrees = 20
edge_lengths = [8, 12]
penumbra = 1

field = pymedphys._mocks.profiles.create_rectangular_field_function([20, 5], edge_lengths, penumbra, degrees)

In [ ]:
x = np.arange(-15, 30, 0.1)
y = np.arange(-15, 15, 0.1)

xx, yy = np.meshgrid(x, y)

zz = field(xx, yy)
plt.pcolormesh(xx, yy, zz)
plt.axis('equal')

In [ ]:
mock_initial_centre = pymedphys._wlutz.findfield.initial_centre(x, y, zz)
mock_initial_centre

In [ ]:
transform = matplotlib.transforms.Affine2D()
transform.translate(*mock_initial_centre)
transform.rotate_deg_around(*mock_initial_centre, -degrees)

interp_size = 20

x_len = 30
y_len = 40

field_x_interp = np.linspace(-interp_size/2, interp_size/2, x_len)
field_y_interp = np.linspace(-interp_size/2, interp_size/2, y_len)

transformed_x = transform @ np.vstack([field_x_interp, np.zeros(x_len), np.ones(x_len)])
transformed_y = transform @ np.vstack([np.zeros(y_len), field_y_interp, np.ones(y_len)])

In [ ]:
transform.get_matrix()

In [ ]:
plt.plot(transformed_x[0], transformed_x[1], label='x-axis')
plt.plot(transformed_y[0], transformed_y[1], label='y-axis')

plt.legend()
plt.axis('equal');

In [ ]:
transformed_x[0:2][1]

In [ ]:
plt.plot(transformed_x[0], transformed_x[1], label='x-axis')
plt.plot(transformed_y[0], transformed_y[1], label='y-axis')
plt.pcolormesh(xx, yy, zz)

plt.legend()
plt.axis('equal')

In [ ]:
x_interp, y_interp = pymedphys._wlutz.createaxis.transform_axis(field_x_interp, field_y_interp, mock_initial_centre, degrees)

plt.plot(x_interp[0], x_interp[1], label='x-axis')
plt.plot(y_interp[0], y_interp[1], label='y-axis')
plt.pcolormesh(xx, yy, zz)

plt.legend()
plt.axis('equal')

In [ ]:
x_profile = pymedphys._mocks.profiles.create_profile_function(0, edge_lengths[0], penumbra)


plt.plot(field_x_interp, x_profile(field_x_interp))
plt.plot(field_x_interp, field(*x_interp))


In [ ]:
y_profile = pymedphys._mocks.profiles.create_profile_function(0, edge_lengths[1], penumbra)

plt.plot(field_y_interp, y_profile(field_y_interp))
plt.plot(field_y_interp, field(*y_interp), '.')

In [ ]:
x_profile(field_x_interp)

In [ ]:
field(*x_interp)

In [ ]:
np.allclose(x_profile(field_x_interp), field(*x_interp))

In [ ]:
np.allclose(y_profile(field_y_interp), field(*y_interp))

In [ ]:
edge_lengths

In [ ]:
img = imageio.imread(image_path)

In [ ]:
assert img.dtype == np.dtype('uint16')
assert np.shape(img) == (1024, 1024)

In [ ]:
shape = np.shape(img)
x_axis = np.arange(-shape[1]/2, shape[1]/2)/4
y_axis = np.arange(-shape[0]/2, shape[0]/2)/4

In [ ]:
plt.imshow(img)
plt.xlim([400, 600])
plt.ylim([600, 400])
plt.colorbar()

In [ ]:
scale_and_flip = 1 - img[::-1,:] / 2**16
plt.contourf(x_axis, y_axis, scale_and_flip, 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
initial_centre = pymedphys._wlutz.findfield.initial_centre(x_axis, y_axis, scale_and_flip)
initial_centre

In [ ]:
interpolation = scipy.interpolate.RectBivariateSpline(x_axis, y_axis, scale_and_flip, kx=1, ky=1)

In [ ]:
scipy.ndimage.measurements.standard_deviation(scale_and_flip)